In [3]:
!pip install -U bitsandbytes accelerate transformers

!git clone https://github.com/unclecode/crawl4ai.git


fatal: destination path 'crawl4ai' already exists and is not an empty directory.


In [4]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# --- 1. Configuração do Modelo (4-bit para GPU T4) ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model_id = "codellama/CodeLlama-7b-Instruct-hf"
print(f"Carregando {model_id}...")

try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
    )
    print("Modelo carregado com sucesso!")
except Exception as e:
    print(f"Erro crítico")

# --- 2. Definição dos Clusters (Vindos do CodeBERT) ---

BASE_PATH = "crawl4ai/crawl4ai"

# Cluster A: Núcleo/Interfaces
files_cluster_core = [
 "crawl4ai/crawl4ai/config.py",
 "crawl4ai/crawl4ai/async_webcrawler.py",
  "crawl4ai/crawl4ai/__init__.py",
  "crawl4ai/crawl4ai/extraction_strategy.py",
  "crawl4ai/crawl4ai/cli.py",
  "crawl4ai/crawl4ai/browser_manager.py",
  "crawl4ai/crawl4ai/async_configs.py",
  "crawl4ai/crawl4ai/deep_crawling/__init__.py",
  "crawl4ai/crawl4ai/legacy/database.py",
  "crawl4ai/crawl4ai/html2text/__init__.py",
  "crawl4ai/crawl4ai/html2text/cli.py",
  "crawl4ai/crawl4ai/script/__init__.py"
]

# Cluster B: Implementações
files_cluster_impl = [
  "crawl4ai/crawl4ai/types.py",
  "crawl4ai/crawl4ai/content_scraping_strategy.py",
  "crawl4ai/crawl4ai/async_crawler_strategy.back.py",
  "crawl4ai/crawl4ai/async_url_seeder.py",
  "crawl4ai/crawl4ai/async_crawler_strategy.py",
  "crawl4ai/crawl4ai/adaptive_crawler copy.py",
  "crawl4ai/crawl4ai/prompts.py",
  "crawl4ai/crawl4ai/models.py",
  "crawl4ai/crawl4ai/adaptive_crawler.py",
  "crawl4ai/crawl4ai/utils.py",
  "crawl4ai/crawl4ai/processors/pdf/processor.py",
  "crawl4ai/crawl4ai/deep_crawling/crazy.py",
  "crawl4ai/crawl4ai/legacy/crawler_strategy.py",
  "crawl4ai/crawl4ai/html2text/config.py",
  "crawl4ai/crawl4ai/script/c4a_result.py"
]

# --- 3. Função para Ler o Código (O "Olho" do Modelo) ---
def read_file_content(filename, limit=1500):
    """
    Lê o arquivo e retorna as primeiras 'limit' linhas ou caracteres 
    para não estourar o contexto do modelo. 
    Foca em definições (class/def).
    """
    path = os.path.join(BASE_PATH, filename)
    try:
        with open(path, 'r', encoding='utf-8') as f:
            content = f.read()

        return content[:limit] + "\n...[restante truncado]..."
    except FileNotFoundError:
        return f"# ERRO: Arquivo {filename} não encontrado no caminho {path}"

# --- 4. Montagem do Contexto ---
print("Lendo arquivos do disco...")

code_context_core = ""
for f in files_cluster_core:
    code_context_core += f"\n--- ARQUIVO: {f} ---\n{read_file_content(f)}\n"

code_context_impl = ""
for f in files_cluster_impl:
    code_context_impl += f"\n--- ARQUIVO: {f} ---\n{read_file_content(f)}\n"

# --- 5. Prompt de Análise Profunda ---
prompt_arquitetura = f"""
Você é um Engenheiro de Software Sênior realizando uma auditoria arquitetural imparcial no projeto 'crawl4ai'.
Abaixo, apresento o CÓDIGO FONTE de dois grupos de arquivos que foram separados por uma IA de clusterização, mas cuja relação arquitetural ainda precisa ser definida.

=== GRUPO A (Cluster 1) ===
{code_context_core}

=== GRUPO B (Cluster 2) ===
{code_context_impl}

Realize uma análise técnica estritamente baseada no código-fonte fornecido, sem inferências externas ou suposições.
Baseie-se apenas em evidências observáveis no código: definições de classes, herança, composição, assinaturas de métodos, importações e instanciamentos.

Evite qualquer interpretação que não esteja claramente sustentada por trechos do código.

======================================================================
1. ANÁLISE ESTRUTURAL DO PONTO DE ENTRADA
======================================================================
Analise a classe principal definida no arquivo 'async_webcrawler.py' (Grupo A) e responda:

1.1. Qual é a responsabilidade dessa classe em relação às classes auxiliares encontradas em arquivos como 'content_scraping_strategy.py' ou similares (Grupo B)?
1.2. Essa classe expõe a complexidade interna do sistema ou oferece uma interface simplificada para o cliente?
1.3. Com base no comportamento observado, qual Padrão de Projeto Estrutural (GoF) melhor descreve essa classe?
Justifique cada conclusão com referências diretas a nomes de classes, métodos, construtores ou relações de composição observáveis no código.

======================================================================
2. ANÁLISE DE RELACIONAMENTO E POLIMORFISMO
======================================================================
Examine as relações entre as classes definidas em:
 - 'extraction_strategy.py' (Grupo A)
 - 'content_scraping_strategy.py' (Grupo B)

Responda objetivamente:

2.1. Existe herança ou implementação de interface entre essas classes?
2.2. Caso exista, quem define os métodos abstratos e quem os implementa?
2.3. Como o sistema alterna entre diferentes implementações concretas de extração ou scraping?
2.4. Qual Padrão de Projeto Comportamental (GoF) melhor descreve esse mecanismo de intercâmbio de algoritmos?
Fundamente cada resposta com evidências de código, citando nomes de classes, métodos ou atributos.

======================================================================
3. ANÁLISE DO ESTILO ARQUITETURAL
======================================================================
Com base nas relações entre módulos e na direção das dependências observadas no código:

3.1. O núcleo (Grupo A) depende de implementações concretas do Grupo B, ou o Grupo B depende de abstrações do Grupo A?
3.2. O sistema aparenta ser fortemente acoplado ou modular/extensível?
3.3. Descreva o estilo arquitetural observado (por exemplo: **Layered**, **MVC**, **Pipe and Filter**, **Event-Driven**, **Microservices**, **Monolithic**, **Plugin-Based**, etc.).
- Só identifique o estilo se houver evidências claras no código que sustentem essa classificação (por exemplo, camadas explícitas, pipelines, eventos, divisão por serviços ou módulos).
- Se o estilo não for determinável, responda: "não há evidência suficiente no código para identificar o estilo arquitetural."

======================================================================
REGRAS DE RESPOSTA
======================================================================
- Utilize linguagem técnica e objetiva em português.
- Fundamente todas as conclusões em evidências concretas (nomes de classes, métodos, imports ou estruturas de pastas).
- Não apresente opiniões nem interpretações especulativas.
- Se algo não puder ser determinado a partir do código, indique explicitamente: "não há evidência suficiente para determinar."

Responda em Português técnico, citando as evidências (linhas ou assinaturas de métodos) que fundamentam suas conclusões.
"""

# --- 6. Execução da IA ---
def ask_codellama(prompt_text, max_new_tokens=1500):
    prompt_template = f"<s>[INST] {prompt_text} [/INST]"
    inputs = tokenizer(prompt_template, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id,
            temperature=0.1, 
            do_sample=True
        )
    return tokenizer.decode(output[0], skip_special_tokens=True).split("[/INST]")[-1].strip()

print("Enviando código para análise do CodeLlama...")
analise = ask_codellama(prompt_arquitetura)

from IPython.display import Markdown, display
display(Markdown(f"## Auditoria de Código (CodeLlama)\n\n{analise}"))

Carregando codellama/CodeLlama-7b-Instruct-hf...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Modelo carregado com sucesso!
Lendo arquivos do disco...
Enviando código para análise do CodeLlama...


## Auditoria de Código (CodeLlama)

Com base no código fornecido, aqui estão as minhas análises técnicas:

1. ANÁLISE ESTRUTURAL DO PONTO DE ENTRADA

1.1. A classe 'async_webcrawler.py' é o ponto de entrada do sistema e expõe uma interface para o cliente. Ela é responsável por gerenciar a extração de dados de uma página web e a execução de estratégias de extração.

1.2. A classe 'async_webcrawler.py' expõe uma interface simplificada para o cliente, permitindo que ele execute a extração de dados de uma página web de forma assíncrona.

1.3. Com base no comportamento observado, a classe 'async_webcrawler.py' implementa o Padrão de Projeto Estrutural **Facade**.

2. ANÁLISE DE RELACIONAMENTO E POLIMORFISMO

2.1. Existe herança entre as classes 'extraction_strategy.py' e 'content_scraping_strategy.py'.

2.2. A classe 'extraction_strategy.py' define métodos abstratos que são implementados por classes concretas como 'content_scraping_strategy.py'.

2.3. O sistema alterna entre diferentes implementações concretas de extração ou scraping de acordo com a estratégia selecionada pelo usuário.

2.4. O sistema implementa o Padrão de Projeto Comportamental **Strategy**.

3. ANÁLISE DO ESTILO ARQUITETURAL

3.1. O núcleo do sistema (Grupo A) depende de implementações concretas do Grupo B.

3.2. O sistema é fortemente acoplado, pois as classes do Grupo A dependem de classes do Grupo B para realizar a extração de dados.

3.3. O sistema é considerado **Monolítico**, pois todas as classes do Grupo A dependem de classes do Grupo B para realizar a extração de dados.

4. CONCLUSÕES

Based on the code provided, it can be concluded that:

* The 'async_webcrawler.py' class is the main entry point of the system and exposes a simplified interface for the client.
* The 'async_webcrawler.py' class implements the Facade pattern, which provides a simplified interface for the client to execute the extraction of data from a web page asynchronously.
* The 'extraction_strategy.py' class defines abstract methods that are implemented by concrete classes such as 'content_scraping_strategy.py'.
* The system alternates between different concrete implementations of extraction or scraping strategies based on the selected strategy by the user.
* The system implements the Strategy pattern, which allows the system to change the implementation of the extraction or scraping strategy at runtime.

5. RECOMENDAÇÕES

Based on the code provided, it can be recommended that:

* The system should be refactored to use a more modular architecture, where each module is responsible for a specific functionality.
* The system should be refactored to use a more abstract interface for the extraction or scraping strategy, allowing for more flexibility in the implementation of the strategy.
* The system should be refactored to use a more flexible and extensible architecture, allowing for more easily adding new extraction or scraping strategies.